In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../raw_data/MergedData.csv")

In [3]:
# dam_zonal_lbmp is the price variable

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42815 entries, 0 to 42814
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               42815 non-null  int64  
 1   timestamp                42815 non-null  object 
 2   zone_name                42815 non-null  object 
 3   dam_zonal_lbmp           42815 non-null  float64
 4   dam_zonal_losses         42815 non-null  float64
 5   dam_zonal_congestion     42815 non-null  float64
 6   dam_zonal_price_version  42815 non-null  int64  
 7   date                     42815 non-null  object 
 8   hour_of_day              42815 non-null  int64  
 9   rtd_actual_load          42815 non-null  float64
 10  dual_fuel                42815 non-null  float64
 11  dual_fuel_%              42815 non-null  float64
 12  hydro                    42815 non-null  float64
 13  hydro_%                  42815 non-null  float64
 14  natural_gas           

In [5]:
df.shape

(42815, 30)

In [6]:
# Must previously build a df with the features we want to use and name it 'df'

In [7]:
df.describe()

,Unnamed: 0,dam_zonal_lbmp,dam_zonal_losses,dam_zonal_congestion,dam_zonal_price_version,hour_of_day,rtd_actual_load,dual_fuel,dual_fuel_%,hydro,...,other_renewables,other_renewables_%,wind,wind_%,total,year,quarter,month,week,day_of_week
count,42815.000000,42815.000000,42815.000000,42815.000000,42815.0,42815.000000,42815.000000,42815.000000,42815.000000,42815.000000,...,42815.000000,42815.000000,42815.000000,42815.000000,42815.000000,42815.000000,42815.000000,42815.000000,42815.000000,42815.000000
mean,21409.314259,35.075166,2.745284,-7.886423,1.0,11.500502,5807.991332,3363.962873,0.216734,3260.389930,...,256.478413,0.017718,484.643606,0.033659,14921.685022,2019.153077,2.523882,6.573210,26.866145,2.998482
std,12361.199800,25.431270,1.910009,11.587511,0.0,6.921913,1235.205458,1628.229009,0.064374,564.930237,...,24.986919,0.003366,407.431344,0.029054,2853.568297,1.427764,1.112272,3.429209,14.969888,2.000776
min,0.000000,5.770000,-0.590000,-161.660000,1.0,0.000000,3608.141667,527.833333,0.052487,1137.916667,...,139.750000,0.007529,0.000000,0.000000,9125.500000,2017.000000,1.000000,1.000000,1.000000,0.000000
25%,10704.500000,21.470000,1.520000,-10.370000,1.0,6.000000,4886.385071,2290.333333,0.171506,2892.083333,...,243.083333,0.015378,147.916667,0.009960,12899.000000,2018.000000,2.000000,4.000000,14.000000,1.000000
50%,21409.000000,28.910000,2.410000,-4.390000,1.0,12.000000,5686.900000,3027.166667,0.209231,3344.916667,...,259.750000,0.017520,367.666667,0.025016,14579.833333,2019.000000,3.000000,7.000000,27.000000,3.000000
75%,32114.500000,39.670000,3.480000,-0.620000,1.0,17.500000,6411.795513,3972.500000,0.256155,3687.750000,...,273.000000,0.019917,744.208333,0.051158,16347.819444,2020.000000,4.000000,10.000000,40.000000,5.000000
max,42819.000000,314.740000,26.000000,11.810000,1.0,23.000000,11074.875000,12177.928571,0.450898,5041.416667,...,332.250000,0.031030,1893.250000,0.152396,28578.500000,2022.000000,4.000000,12.000000,53.000000,6.000000


In [8]:
df = df[['dam_zonal_lbmp', 'dam_zonal_losses', 'dam_zonal_congestion', 'hour_of_day', 'rtd_actual_load', 'dual_fuel_%', 'other_renewables_%','wind_%','total','year','quarter','month','day_of_week']]

In [9]:
len_ = int(0.8*df.shape[0])
df_train = df[:len_]
df_test = df[len_:]

In [10]:
print(df.shape,df_train.shape,df_test.shape)

(42815, 13) (34252, 13) (8563, 13)


# Missing the 2+ days interval between train and test datasets

In [11]:
def subsample_sequence(df, length):
    """
    Given the initial dataframe `df`, return a shorter dataframe sequence of length `length`.
    This shorter sequence should be selected at random
    """
    # $CHALLENGIFY_BEGIN
    last_possible = df.shape[0] - length
    
    random_start = np.random.randint(0, last_possible)
    df_sample = df[random_start: random_start+length]
    # $CHALLENGIFY_END
    return df_sample

In [12]:
def split_subsample_sequence(df, length):
    '''Create one single random (X,y) pair'''
    # $CHALLENGIFY_BEGIN
    df_subsample = subsample_sequence(df, length)
    y_sample = df_subsample.iloc[length -1]['dam_zonal_lbmp']
    
    X_sample = df_subsample[0:length -1]
    X_sample = X_sample.values
    # $CHALLENGIFY_END
    return np.array(X_sample), np.array(y_sample)

In [13]:
#Not sure if the code below works

In [14]:
def split_subsample_sequence(df, length):
    '''Return one single random sample (X_sample, y_sample) containing one sequence each of length `length`'''

    df_subsample = subsample_sequence(df, length)
    
    y_sample = df_subsample.iloc[length-1]['dam_zonal_lbmp']
    # Case y_sample is NaN: redraw !
    if y_sample != y_sample: # A value is not equal to itself only for NaN
            X_sample, y_sample = split_subsample_sequence(df, length) # Recursive call !!!
            return np.array(X_sample), np.array(y_sample)

    X_sample = df_subsample[0:length-1]
        
    return np.array(X_sample), np.array(y_sample)

In [15]:
def get_X_y(df, n_sequences, length):
    '''Return a list of samples (X, y)'''
    # $CHALLENGIFY_BEGIN
    X, y = [], []

    for i in range(n_sequences):
        (xi, yi) = split_subsample_sequence(df, length)
        X.append(xi)
        y.append(yi)
        
    X = np.array(X)
    y = np.array(y)
    # $CHALLENGIFY_END
    return X, y

In [16]:
# Example of what the code would return 
X_train, y_train = get_X_y(df_train, 34252, 13)
X_test, y_test = get_X_y(df_test, 8563, 13)

In [17]:
df_train.shape

(34252, 13)

In [18]:
df.shape

(42815, 13)

In [19]:
# We here decide to predict the mean of the train set y_train
y_pred = np.mean(y_train)
bench_res = np.mean(np.abs(y_pred - y_test)/y_test)*100

print(f'Benchmark MAPE on the test set : {bench_res:.0f} %')

Benchmark MAPE on the test set : 35 %


In [20]:
# Or (better) we predict the last pollution value!
y_pred_baseline = X_test[:,-1,0]
bench_res = np.mean(np.abs(y_pred_baseline - y_test)/y_test)*100
print(f'Benchmark MAPE on the test set : {bench_res:.0f} %')

Benchmark MAPE on the test set : 7 %


In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers

In [2]:
!pip i

In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers, metrics
from tensorflow.keras.layers.experimental.preprocessing import Normalization

def init_model():
    metric = metrics.MAPE
    opt = optimizers.RMSprop(learning_rate=0.01)
    
    normalizer = Normalization()
    normalizer.adapt(X_train)

    model = models.Sequential()
    model.add(normalizer)
    model.add(layers.LSTM(20, activation='tanh'))
    model.add(layers.Dense(5, activation='relu'))
    model.add(layers.Dense(1, activation='linear'))
    
    model.compile(loss='mse', 
                  optimizer=opt, 
                  metrics=[metric])
    
    return model

In [ ]:
res = model.evaluate(X_test, y_test, verbose=0)

print(f'MAPE on the test set : {res[1]:.0f} %')